In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from pyaedt.hfss import Hfss

solution_type = "Modal" # "Eigenmode", "Modal"

use_antenna = False
do_chip_angle_sweep = False
do_outer_radius_sweep = False
do_coax_height_sweep = False

hfss = Hfss(
    projectname="test.aedt",
    designname="A1",
    solution_type=solution_type
)

PyAEDT WARNING: Argument `designname` is deprecated for method `__init__`; use `design` instead.
PyAEDT WARNING: Argument `projectname` is deprecated for method `__init__`; use `project` instead.
PyAEDT INFO: Python version 3.11.13 | packaged by conda-forge | (main, Jun  4 2025, 14:39:58) [MSC v.1943 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.17.4.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\ymino\AppData\Local\Temp\15\pyaedt_ymino_68018a56-53de-4540-93d9-bccbd0572100.log is enabled.
PyAEDT INFO: Log on AEDT is disabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT ERROR: A(n) <class 'psutil.AccessDenied'> error occurred while retrieving information for the active AEDT sessions: (pid=11984, name='ansysedt.exe')
PyAEDT ERROR: A(n) <class 'psutil.AccessDenied'> error occurred while retrieving information for the active AEDT s

In [2]:
hfss["box_height"] = "38mm"
hfss["box_length"] = "15mm"
hfss["$outer_radius"] = "5mm"
hfss["outer_height"] = "38mm"
hfss["coax_radius"] = "1.5mm"
hfss["$coax_height"] = "38mm" #"9mm"
hfss["antenna_height"] = "9mm"
hfss["antenna_radius"] = "1.5mm"
hfss["chip_gap"] = "20um"
hfss["chip_width"] = "250um"
hfss["chip_height"] = "400um"
hfss["$chip_theta"] = "0deg"

In [3]:
box_config = dict(
    position = ["-box_length/2", "-box_length/2", "outer_height - box_height"],
    dimensions_list = ["box_length", "box_length", "box_height"],
    name = "box",
    matname = "copper"
)
if use_antenna:
    box_config.update( dimensions_list = ["box_length", "box_length", "box_height + antenna_height"] )

box_object = hfss.modeler.create_box( **box_config )


_, vacuum_name = hfss.modeler.duplicate_around_axis(box_object, "Z", angle = 0)
vacuum_object = hfss.modeler.get_object_from_name(vacuum_name[0])
vacuum_object.name = "boundary"
hfss.assign_material("boundary", "vacuum")

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec
PyAEDT WARNING: Argument `position` is deprecated for method `create_box`; use `origin` instead.
PyAEDT WARNING: Argument `dimensions_list` is deprecated for method `create_box`; use `sizes` instead.
PyAEDT WARNING: Argument `matname` is deprecated for method `create_box`; use `material` instead.
PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


True

In [4]:
cylinder_config = dict(
    cs_axis="Z" ,
    position=[0, 0, 0], 
    radius="$outer_radius", 
    height="outer_height", 
    name="outer", 
    matname="vacuum"
)

cylinder_object = hfss.modeler.create_cylinder( **cylinder_config )

PyAEDT WARNING: Argument `cs_axis` is deprecated for method `create_cylinder`; use `orientation` instead.
PyAEDT WARNING: Argument `position` is deprecated for method `create_cylinder`; use `origin` instead.
PyAEDT WARNING: Argument `matname` is deprecated for method `create_cylinder`; use `material` instead.


In [5]:
box_object.subtract(cylinder_object, keep_originals=False)

PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


In [6]:
cylinder_config = dict(
    cs_axis="Z" ,
    position=[0, 0, 0], 
    radius="coax_radius", 
    height="$coax_height", 
    name="coax", 
    matname="copper"
)

coax_object = hfss.modeler.create_cylinder( **cylinder_config )

PyAEDT WARNING: Argument `cs_axis` is deprecated for method `create_cylinder`; use `orientation` instead.
PyAEDT WARNING: Argument `position` is deprecated for method `create_cylinder`; use `origin` instead.
PyAEDT WARNING: Argument `matname` is deprecated for method `create_cylinder`; use `material` instead.


In [7]:
box_object.unite(coax_object)

PyAEDT INFO: Parsing design objects. This operation can take time
PyAEDT INFO: Refreshing bodies from Object Info
PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec
PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec
PyAEDT INFO: Union of 2 objects has been executed.


## Create antenna

In [8]:
if use_antenna:
    antenna_config = dict(
        cs_axis="Z" ,
        position=[0, 0, "outer_height"], 
        radius="$outer_radius", 
        height="antenna_height", 
        name="antenna", 
        matname="vacuum"
    )

    antenna_object = hfss.modeler.create_cylinder( **antenna_config )
    box_object.subtract(antenna_object, keep_originals=False)

    antenna_config = dict(
        cs_axis="Z" ,
        position=[0, 0, "outer_height"], 
        radius="coax_radius", 
        height="antenna_height", 
        name="core", 
        matname="copper"
    )

    core_object = hfss.modeler.create_cylinder( **antenna_config )

## Create chip

In [9]:
chip_coord = hfss.modeler.create_coordinate_system(origin = [0, "0.5*(coax_radius + $outer_radius)","0.8*$coax_height"], name = "chip_coord")
chip_coord.set_as_working_cs()
cap1 = hfss.modeler.create_rectangle(origin = ["-0.5*chip_width", "0.5*chip_gap", 0], sizes = ["chip_width", "chip_height"], name = "cap1", orientation="XY")
cap2 = hfss.modeler.create_rectangle(origin = ["-0.5*chip_width", "-0.5*chip_gap - chip_height", 0], sizes =  ["chip_width", "chip_height"], name = "cap2", orientation="XY")

In [10]:
## Only for newer pyaedt versions ?
hfss.assign_perfect_e("box")
hfss.assign_perfect_e("cap1")
hfss.assign_perfect_e("cap2")
if use_antenna:
    hfss.assign_perfect_e("core")

PyAEDT INFO: Boundary Perfect E PerfectE_box has been created.
PyAEDT INFO: Boundary Perfect E PerfectE_cap1 has been created.
PyAEDT INFO: Boundary Perfect E PerfectE_cap2 has been created.


## Create ports

In [11]:
# faces = vacuum_object.faces
# top_face = max(faces, key=lambda f: f.center[2])

In [12]:
hfss.modeler.set_working_coordinate_system("Global")
if use_antenna:
    port_in = hfss.modeler.create_circle(origin = [0, 0, "outer_height + antenna_height"], radius = "$outer_radius", name = "port_in", orientation="XY")
else:
    port_in = hfss.modeler.create_circle(origin = [0, 0, "outer_height"], radius = "$outer_radius", name = "port_in", orientation="XY")
    readout = hfss.modeler.create_circle(origin = [0, 0, 0], radius = "$outer_radius", name = "readout", orientation="XY")

hfss.wave_port(assignment="port_in", name = "Port_in")
hfss.wave_port(assignment="readout", name = "Readout")

hfss.modeler.set_working_coordinate_system("chip_coord")
port_out = hfss.modeler.create_rectangle(origin = ["-0.5*chip_gap","-0.5*chip_gap" ], sizes=["chip_gap","chip_gap"], name = "port_out", orientation="XY")
hfss.lumped_port(assignment="port_out", integration_line = hfss.AxisDir.YNeg, name="Port_out")

PyAEDT INFO: Boundary Wave Port Port_in has been created.
PyAEDT INFO: Boundary Wave Port Readout has been created.
PyAEDT INFO: Boundary Lumped Port Port_out has been created.


## Rotate chip

In [13]:
hfss.modeler.set_working_coordinate_system("chip_coord")
hfss.modeler.rotate(assignment=["cap1","cap2","port_out"], axis="X", angle="$chip_theta")

True

In [ ]:
# if solution_type=="Modal":
#     hfss.create_open_region(Frequency="1GHz")

if solution_type=="Modal":

    setup = hfss.create_setup("MySetup") 
    setup.create_frequency_sweep(
        unit="GHz",
        name="Sweep1",
        start_frequency=0.0,
        stop_frequency=20.0,
        sweep_type="Interpolating",
    )

PyAEDT INFO: Parsing C:/HFSS/ymino/test.aedt.
PyAEDT INFO: File C:/HFSS/ymino/test.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: aedt file load time 0.057840824127197266
PyAEDT INFO: Linear count sweep Sweep1 has been correctly created


In [15]:
if solution_type=="EigenMode":
    setup = hfss.create_setup("MySetup") 
    setup.props["Frequency"]= "50MHz" 
    setup.props["MaximumPasses"]=10

In [16]:
if not (do_chip_angle_sweep or do_outer_radius_sweep or do_coax_height_sweep):
    hfss.analyze_setup(name = "MySetup", cores=8)

PyAEDT INFO: Key Desktop/ActiveDSOConfigurations/HFSS correctly changed.
PyAEDT INFO: Solving design setup MySetup
PyAEDT INFO: Design setup MySetup solved correctly in 0.0h 1.0m 51.0s


In [17]:
if do_chip_angle_sweep:
    sweep = hfss.parametrics.add(
        variable="$chip_theta",
        start_point=0,
        end_point=90,
        step=10,
        name="ChipAngle"
    )

    sweep.analyze(cores = 8)

In [18]:
if do_outer_radius_sweep:
    radius_sweep = hfss.parametrics.add(
        variable="$outer_radius",
        start_point=5,
        end_point=15,
        step=6,
        name="Radius"
    )
    radius_sweep.analyze(cores = 8)

In [19]:
if do_coax_height_sweep:
    height_sweep = hfss.parametrics.add(
        variable="$coax_height",
        start_point=9,
        end_point=38,
        step=3,
        name="Coax_height"
    )
    height_sweep.analyze(cores = 8)

## Produce Report

In [20]:
report_config = dict(
    expressions=["db(S12)"], 
    plot_name="S-parameter", 
    variations={
        "Freq": ["All"],
    }
)

if do_chip_angle_sweep:
    report_config["variations"]["$chip_theta"] = ["0deg","90deg"] # ["All"]
if do_outer_radius_sweep:
    report_config["variations"]["$outer_radius"] = ["All"]
if do_coax_height_sweep:
    report_config["variations"]["$coax_height"] = ["All"]

hfss.post.create_report( **report_config )

PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 0sec
PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


In [ ]:
# traces_to_plot = hfss.get_traces_for_plot()
# report = hfss.post.create_report(traces_to_plot)  # Creates a report in HFSS
# solution = report.get_solution_data()
# solution.plot(solution.expressions[1])  # Matplotlib axes object.

NameError: name 'hfss' is not defined

In [17]:
## Get intrinsic information (First you need the solution frequency)

intrinsics = hfss.setups[0].default_intrinsics
print(intrinsics)
# intrinsics["Freq"] = hfss.setups[0].properties["Solution Freq"]

{'Freq': '5GHz', 'Phase': '0deg'}


In [ ]:
## Plot field (Currently disabled)
# cutlist = ["Global:YZ"]
# setup_name = hfss.existing_analysis_sweeps[0]
# #setup_name = "Sweep1"
# quantity_name = "Mag_E"
# intrinsic = {"Freq": "5GHz", "Phase": "0deg"}
# hfss.logger.info("Generating the image")
# plot_obj = hfss.post.plot_field(
#         quantity="Mag_E",
#         assignment=cutlist,
#         plot_type="CutPlane",
#         setup=setup_name,
#         intrinsics=intrinsic
#     )

PyAEDT INFO: Generating the image
PyAEDT INFO: Active Design set to A1


ERROR:root:failed to get wglChoosePixelFormatARB
ERROR:root:failed to get valid pixel format.


: 

In [18]:
#field_plot = hfss.post.create_fieldplot_volume( ["boundary"],"Mag_E", plot_name="boundary_Mag")
plane_plot = hfss.post.create_fieldplot_cutplane("Global:YZ", "Mag_E", plot_name = "plane_Mag")

PyAEDT INFO: Active Design set to A1
